## 2019-03-28 Making Dataframes
Today has been spent turning the large .sd file into a
series of more manageable dataframes. 
The script /src/data/make_panda_dataframes.py
will do so and take about 20 minutes on my laptop.

In [2]:
import numpy as np
import sklearn
from rdkit import Chem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG, IFrame
import gzip
import os
import pickle
import pandas as pd

In [3]:
with open("../data/processed/curated_set_with_publication_year.pd.pkl", "rb") as infile:
    df = pickle.load(infile)


In [4]:
for index, item in enumerate(df.dtypes):
    print(index, item)

0 float64
1 float64
2 float64
3 float64
4 int64
5 object
6 float64
7 int64
8 int64
9 int64
10 bool
11 float64
12 object
13 bool
14 bool
15 bool
16 bool
17 object
18 int64
19 int64
20 float64
21 object
22 int64
23 object
24 object
25 bool
26 bool
27 int64
28 object
29 object
30 object
31 object
32 object


Extract the numerical columns from the table, and put them into a numpy array so we can try to map them.


In [5]:
numerical_cols = [df.columns[pos] for pos, item in enumerate(df.dtypes) if item in [np.float64, np.int64]]
print(numerical_cols)
data = df[numerical_cols].values
print(data.shape)

['BIOACT_PCHEMBL_VALUE', 'CMP_ACD_LOGD', 'CMP_ACD_LOGP', 'CMP_ALOGP', 'CMP_AROMATIC_RINGS', 'CMP_FULL_MWT', 'CMP_HBA', 'CMP_HBD', 'CMP_HEAVY_ATOMS', 'CMP_LOGP', 'CMP_NUM_ALERTS', 'CMP_NUM_RO5_VIOLATIONS', 'CMP_PSA', 'CMP_RTB', 'DOC_YEAR']
(75000, 15)


In [11]:
import kmapper as km
from kmapper import jupyter
from sklearn import cluster
mapper = km.KeplerMapper(verbose=1)
projected_data = mapper.project(data, projection="sum")
graph = mapper.map(projected_data,
                   data,
                   cover=km.Cover(n_cubes=5, perc_overlap=0.75),
                   clusterer=cluster.AgglomerativeClustering(n_clusters=100,
                                                             affinity="cosine"))
# Visualize it
mapper.visualize(graph, path_html="map-dataframe-test.html",
                 title="Map Dataframe Test")
IFrame("map-dataframe-test.html", 800, 600)

KeplerMapper(verbose=1)
..Projecting on data shaped (75000, 15)

..Projecting data using: sum

..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))

Mapping on data shaped (75000, 15) using lens shaped (75000, 1)

Creating 5 hypercubes.

Created 0 edges and 0 nodes in 0:00:00.026833.


/home/matthew-bailey/kepler-mapper/kmapper/cover.py:233: RuntimeWarning: invalid value encountered in greater_equal
  entries = (data[:, self.di_] >= lowerbounds) & (
/home/matthew-bailey/kepler-mapper/kmapper/cover.py:234: RuntimeWarning: invalid value encountered in less_equal
  data[:, self.di_] <= upperbounds


Exception: Visualize requires a mapper with more than 0 nodes. 
It is possible that the constructed mapper could have been constructed with bad parameters. This can occasionally happens when using the default clustering algorithm. Try changing `eps` or `min_samples` in the DBSCAN clustering algorithm.